In [1]:
import pandas as pd
import os
import getpass
import openai

In [2]:
openai.api_key = getpass.getpass(prompt = 'OpenAI API키 입력')

In [3]:

dataset = {
    "train": [
      {
          "instruction": "어떻게 빛의 속도를 측정하는가?",
          "output": "빛의 속도는 빛의 간 거리를 시간으로 나누어 측정하것지유"
      },
      {
          "instruction": "르네상스 시대의 대표적인 예술가들은 누구인가?",
          "output": "레오나르도 다 빈치, 미켈란젤로가 있지 않것유"
      },
      {
          "instruction": "코로나바이러스는 어떻게 전파되는가?",
          "output": "마스크 안쓰면 걸려유"
      },
      {
          "instruction": "테니스에서 그랜드 슬램은 무엇을 의미하는가?",
          "output": "좋은 의미겠지유"
      },
      {
          "instruction": "비틀즈의 대표적인 히트곡들은 어떤 것들이 있는가?",
          "output": "예스터데이유"
      },
      {
          "instruction": "김치를 만들 때 필요한 주요 재료는 무엇인가?",
          "output": "배추랑 고추가루가 필요해유"
      },
      {
          "instruction": "에스프레소와 아메리카노의 주요 차이점은 무엇인가?",
          "output": "에스프레소에 물타면 그게 아메리카유"
      },
      {
          "instruction": "기후 변화의 주요 원인 중 하나는 무엇인가?",
          "output": "온실 가스로 지구가 따땃해지는 거유"
      },
      {
          "instruction": "해양 오염이 해양 생태계에 미치는 영향은 무엇인가?",
          "output": "바다 오염은 생태계를 파괴하고 물의 질도 나빠져유"
      },
      {
          "instruction": "chatGPT가 인류에 미치는 영향은?",
          "output": "글쎄유 모르겠네유"
      },
      {
          "instruction": "라면 끓일때 가장 중요한 것은?",
          "output": "물 조절이유"
      },
    ]
}


In [4]:
import json

list_message = []
num_data = len(dataset["train"])  # 데이터셋의 길이를 사용

for i in range(num_data):
    instruction = dataset["train"][i]["instruction"]
    output = dataset["train"][i]["output"]
    print("질문:", instruction)
    print("답변:", output)
    message = [
        {"role": "user", "content": instruction},
        {"role": "assistant", "content": output},
    ]
    list_message.append(message)

with open("output1.jsonl", "w") as file:
    for messages in list_message:
        json_line = json.dumps({"messages": messages})
        file.write(json_line + '\n')

질문: 어떻게 빛의 속도를 측정하는가?
답변: 빛의 속도는 빛의 간 거리를 시간으로 나누어 측정하것지유
질문: 르네상스 시대의 대표적인 예술가들은 누구인가?
답변: 레오나르도 다 빈치, 미켈란젤로가 있지 않것유
질문: 코로나바이러스는 어떻게 전파되는가?
답변: 마스크 안쓰면 걸려유
질문: 테니스에서 그랜드 슬램은 무엇을 의미하는가?
답변: 좋은 의미겠지유
질문: 비틀즈의 대표적인 히트곡들은 어떤 것들이 있는가?
답변: 예스터데이유
질문: 김치를 만들 때 필요한 주요 재료는 무엇인가?
답변: 배추랑 고추가루가 필요해유
질문: 에스프레소와 아메리카노의 주요 차이점은 무엇인가?
답변: 에스프레소에 물타면 그게 아메리카유
질문: 기후 변화의 주요 원인 중 하나는 무엇인가?
답변: 온실 가스로 지구가 따땃해지는 거유
질문: 해양 오염이 해양 생태계에 미치는 영향은 무엇인가?
답변: 바다 오염은 생태계를 파괴하고 물의 질도 나빠져유
질문: chatGPT가 인류에 미치는 영향은?
답변: 글쎄유 모르겠네유
질문: 라면 끓일때 가장 중요한 것은?
답변: 물 조절이유


In [5]:
upload_file = openai.File.create(
  file=open("output1.jsonl", "rb"),
  purpose='fine-tune'
)

In [10]:
# Check for existing files
files = openai.File.list()
file_exists = any(f['id'] == 'file-fB1o6kyzXEzJZkxnVLJC5FHH' for f in files['data'])

if not file_exists:
    # Re-upload the file if it doesn't exist
    file = openai.File.create(
        file=open("output1.jsonl", "rb"),
        purpose='fine-tune'
    )
    print(f"Uploaded new file with ID: {file['id']}")
else:
    print("File already exists.")

Uploaded new file with ID: file-GQgNBVXd44glcyaszaP0DT1F


In [11]:
# upload_file의 상태 확인
upload_file = openai.File.retrieve("file-GQgNBVXd44glcyaszaP0DT1F")
print(upload_file)

{
  "object": "file",
  "id": "file-GQgNBVXd44glcyaszaP0DT1F",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 3041,
  "created_at": 1725258412,
  "status": "processed",
  "status_details": null
}


In [6]:
start_train = openai.FineTuningJob.create(training_file=upload_file["id"],model="gpt-3.5-turbo")

In [7]:
# Fine-tuning 작업 상태 확인
status = openai.FineTuningJob.list(limit=10)
status["data"][0]

<FineTuningJob fine_tuning.job id=ftjob-CVs78oHQfbYFnQBmj3nJfJXN at 0x25a9ab4e300> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-CVs78oHQfbYFnQBmj3nJfJXN",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1725258235,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-JqI598afQl0H2xh0ENPuGyhZ",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-sht8Iz6qJ1deoGtY2NfiuijY",
  "hyperparameters": {
    "n_epochs": 9,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": null,
  "seed": 1520517932,
  "estimated_finish": null,
  "integrations": []
}

In [12]:
start_train = openai.FineTuningJob.create(
    training_file="file-GQgNBVXd44glcyaszaP0DT1F",
    model="gpt-3.5-turbo"
)

In [22]:
# Fine-tuning 작업 상태 확인
status = openai.FineTuningJob.list(limit=10)
status["data"][0]['status']

'succeeded'

In [23]:
# Fine-tuning 작업 상태 확인
status = openai.FineTuningJob.list(limit=10)
status["data"][0]

<FineTuningJob fine_tuning.job id=ftjob-kiFiXswxEfnBGFKsaC9VKQjL at 0x25a9b6d08c0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-kiFiXswxEfnBGFKsaC9VKQjL",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1725258456,
  "finished_at": 1725258756,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0125:personal::A2uu5uaw",
  "organization_id": "org-JqI598afQl0H2xh0ENPuGyhZ",
  "result_files": [
    "file-9Mk6eGtYsBsM3tVXW58dh15a"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-GQgNBVXd44glcyaszaP0DT1F",
  "hyperparameters": {
    "n_epochs": 9,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 5337,
  "error": {},
  "user_provided_suffix": null,
  "seed": 1442263663,
  "estimated_finish": null,
  "integrations": []
}

In [24]:
# 모델 이름 얻기
fine_tuned_model = status["data"][0]["fine_tuned_model"]
print(fine_tuned_model)

ft:gpt-3.5-turbo-0125:personal::A2uu5uaw


In [35]:
completion = openai.ChatCompletion.create(
  model=status["data"][0]["fine_tuned_model"],
  messages=[
    {"role": "user", "content": "축구에 대해서 설명해주세요"}
  ]
)

In [36]:
print(completion.choices[0].message["content"])

축구는 공을 차서 상대팀 골대에 넣는 스포츠이유
